In [1]:
import glfw
import numpy as np
from math import sin, cos
from OpenGL.GL import *
from OpenGL.GL.shaders import compileProgram, compileShader
from OpenGL.GLU import *
from OpenGL.GLUT import *
from plyfile import PlyData, PlyElement
import pyrr
import math
from PIL import Image
from camera import Camera
import random

In [2]:
vertex_src = """
# version 330

layout(location = 0) in vec3 a_position;
layout(location = 1) in vec3 a_color;

uniform mat4 model;
uniform mat4 projection;
uniform mat4 view;

out vec3 v_color;

void main()
{
    gl_Position = projection * view * model * vec4(a_position, 1.0);
    v_color = a_color;
}
"""

fragment_src = """
# version 330

in vec3 v_color;
out vec4 out_color;

void main()
{
    out_color = vec4(v_color, 1.0);
}
"""

In [3]:
with open('c:/Users/alpha02/Documents/PLY-FACE-DATA/male_face_c10s10e5.ply', 'rb') as f:
    face = PlyData.read(f)
#顶点 x y z 色彩 R G B A 数值为1
face_vertices=[]
vertices_xyz=[]
RGB_color=[]
for i in range(len(face.elements[0])):
    vertices_xyz.append(face.elements[0].data[i][0])
    vertices_xyz.append(face.elements[0].data[i][1])
    vertices_xyz.append(face.elements[0].data[i][2])
    RGB_color.append(face.elements[0].data[i][3])
    RGB_color.append(face.elements[0].data[i][4])
    RGB_color.append(face.elements[0].data[i][5])
vertices_xyz = np.array(vertices_xyz, dtype=np.float32)    
RGB_color = np.array(RGB_color, dtype=np.float32)
#顶点归一化及中心化
Min = np.min(vertices_xyz)
Max = np.max(vertices_xyz)
vertices_xyz = (vertices_xyz - Min) / (Max - Min)
Mean = np.mean(vertices_xyz)
vertices_xyz = vertices_xyz-Mean
#色彩归一化
Min = np.min(RGB_color)
Max = np.max(RGB_color)
RGB_color = (RGB_color-Min) / (Max - Min)
for i in range(len(face.elements[0])):
    face_vertices.append(vertices_xyz[0+i*3])
    face_vertices.append(vertices_xyz[1+i*3])
    face_vertices.append(vertices_xyz[2+i*3])
    face_vertices.append(RGB_color[0+i*3])
    face_vertices.append(RGB_color[1+i*3])
    face_vertices.append(RGB_color[2+i*3])
face_vertices = np.array(face_vertices, dtype=np.float32)
#顶点索引，三点一组
face_indices=[]
for i in range(len(face.elements[1])):
    face_indices.append(face.elements[1].data[i][0][0])
    face_indices.append(face.elements[1].data[i][0][1])
    face_indices.append(face.elements[1].data[i][0][2])
face_indices = np.array(face_indices, dtype=np.uint32)

In [4]:
with open('c:/Users/alpha02/Documents/PLY-FACE-DATA/sphere_without_color.ply', 'rb') as s:
    sphere = PlyData.read(s)
#顶点 x y z 色彩 R G B  取颜色为蓝色 0 170 255 归一化 0 0.67 1
sphere_vertices = []
obj_size=0.05 #设定球体大小
for i in range(len(sphere.elements[0])):
    sphere_vertices.append(sphere.elements[0].data[i][0]*obj_size)
    sphere_vertices.append(sphere.elements[0].data[i][1]*obj_size)
    sphere_vertices.append(sphere.elements[0].data[i][2]*obj_size)
    sphere_vertices.append(0)
    sphere_vertices.append(0.67)
    sphere_vertices.append(1)
sphere_vertices = np.array(sphere_vertices, dtype=np.float32)
#顶点索引，三点一组
sphere_indices = []
for i in range(len(sphere.elements[1])):
    sphere_indices.append(sphere.elements[1].data[i][0][0])
    sphere_indices.append(sphere.elements[1].data[i][0][1])
    sphere_indices.append(sphere.elements[1].data[i][0][2])
sphere_indices = np.array(sphere_indices, dtype=np.uint32)    

In [5]:
#物体随机坐标
coordinates = []
obj_size=1 #设定球体大小
for i in range(len(sphere.elements[0])):
    pos=[]
    pos.append(sphere.elements[0].data[i][0]*obj_size )
    pos.append(sphere.elements[0].data[i][1]*obj_size)
    pos.append(sphere.elements[0].data[i][2]*obj_size)
    coordinates.append(pos)
coordinates = np.array(coordinates, dtype=np.float32)

In [6]:
cam = Camera()
WIDTH, HEIGHT = 768, 768
lastX, lastY = WIDTH / 2, HEIGHT / 2
first_mouse = True
left, right, forward, backward = False, False, False, False

# the keyboard input callback
def key_input_clb(window, key, scancode, action, mode):
    global left, right, forward, backward
    if key == glfw.KEY_ESCAPE and action == glfw.PRESS:
        glfw.set_window_should_close(window, True)
    if key == glfw.KEY_W and action == glfw.PRESS:
        forward = True
    elif key == glfw.KEY_W and action == glfw.RELEASE:
        forward = False
    if key == glfw.KEY_S and action == glfw.PRESS:
        backward = True
    elif key == glfw.KEY_S and action == glfw.RELEASE:
        backward = False
    if key == glfw.KEY_A and action == glfw.PRESS:
        left = True
    elif key == glfw.KEY_A and action == glfw.RELEASE:
        left = False
    if key == glfw.KEY_D and action == glfw.PRESS:
        right = True
    elif key == glfw.KEY_D and action == glfw.RELEASE:
        right = False
    # if key in [glfw.KEY_W, glfw.KEY_S, glfw.KEY_D, glfw.KEY_A] and action == glfw.RELEASE:
    # left, right, forward, backward = False, False, False, False

# do the movement, call this function in the main loop
def do_movement():
    if left:
        cam.process_keyboard("LEFT", 0.005)
    if right:
        cam.process_keyboard("RIGHT", 0.005)
    if forward:
        cam.process_keyboard("FORWARD", 0.005)
    if backward:
        cam.process_keyboard("BACKWARD", 0.005)

# the mouse position callback function
def mouse_look_clb(window, xpos, ypos):
    global first_mouse, lastX, lastY
    if first_mouse:
        lastX = xpos
        lastY = ypos
        first_mouse = False
    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos
    cam.process_mouse_movement(xoffset, yoffset)
    
#the mouse click callback function
def mouse_button_click(window, button, action, mods):
    if button == glfw.MOUSE_BUTTON_LEFT:
        data = glReadPixels (0, 0, WIDTH, HEIGHT, GL_RGB,  GL_UNSIGNED_BYTE)
        image = Image.new ("RGB", (WIDTH, HEIGHT), (0, 0, 0))
        image.frombytes (data)
        image = image.crop ((0,0,WIDTH, HEIGHT))
        image = image.transpose(Image.FLIP_TOP_BOTTOM)
        image.save ('saveImage.png')
        
#random pick coordinates
def rnd_sphere_coordinate(coordinate):
    return coordinate[random.randint(0,len(coordinate)-1)]

In [7]:
def window_resize(window, width, height):
    glViewport(0, 0, width, height)
    
# initializing glfw library
if not glfw.init():
    raise Exception("glfw can not be initialized!")
# creating the window
window = glfw.create_window(WIDTH, HEIGHT, "My OpenGL window", None, None)
# check if window was created
if not window:
    glfw.terminate()
    raise Exception("glfw window can not be created!")
# set window's position
glfw.set_window_pos(window, 200, 200)
# set the callback function for window resize
glfw.set_window_size_callback(window, window_resize)
# make the context current
glfw.make_context_current(window)

#设置摄像头移动调用
# set the mouse position callback
glfw.set_cursor_pos_callback(window, mouse_look_clb)
# set the keyboard input callback
glfw.set_key_callback(window, key_input_clb)
# capture the mouse cursor
glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED)
# capture the mouse click
glfw.set_mouse_button_callback(window,mouse_button_click)

#initial shader
shader = compileProgram(compileShader(vertex_src, GL_VERTEX_SHADER),compileShader(fragment_src, GL_FRAGMENT_SHADER))
glUseProgram(shader)
proj_loc = glGetUniformLocation(shader, "projection")
model_loc = glGetUniformLocation(shader, "model")
view_loc = glGetUniformLocation(shader, "view")

# Vertex Buffer Object 为物体分配顶点对象缓存 存储所有顶点信息，但不包含索引信息。VAO中包含VBO和EBO
VBO = glGenBuffers(2)
# Element Buffer Object 为物体索引信息分配缓存 只存储索引信息每个物体一个缓存
EBO = glGenBuffers(2)

#设定映射
projection = pyrr.matrix44.create_perspective_projection_matrix(100, 1024/1024, 0.1, 100)#第一个值设置大小，值越大物体越小
glUniformMatrix4fv(proj_loc, 1, GL_FALSE, projection)

glEnable(GL_DEPTH_TEST) #图像深度三维化
glClearColor(0.68, 0.68, 0.68, 1)#设置背景色

# the main application loop
while not glfw.window_should_close(window):
    glfw.poll_events()
    
    do_movement()
    
    xyz = rnd_sphere_coordinate(coordinates)
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    #绑定照相机
    view = cam.get_view_matrix()
    glUniformMatrix4fv(view_loc, 1, GL_FALSE, view)
    
    #载入面孔
    glBindBuffer(GL_ARRAY_BUFFER, VBO[0])
    glBufferData(GL_ARRAY_BUFFER, face_vertices.nbytes, face_vertices, GL_STATIC_DRAW)
    glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, EBO[0])
    glBufferData(GL_ELEMENT_ARRAY_BUFFER,face_indices.nbytes, face_indices, GL_STATIC_DRAW)
    #enable vertices
    glEnableVertexAttribArray(0)
    glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 24, ctypes.c_void_p(0))#从第0个数据开始，元素个数为3
    #enable color
    glEnableVertexAttribArray(1)
    glVertexAttribPointer(1, 3, GL_FLOAT, GL_FALSE, 24, ctypes.c_void_p(12))#从第1个数据开始，元素个数为3
    face_pos =  pyrr.matrix44.create_from_translation(pyrr.Vector3([0, 0, 0])) #定义物体世界坐标 0, 1, -3
    glUniformMatrix4fv(model_loc, 1, GL_FALSE, face_pos)
    glDrawElements(GL_TRIANGLES, len(face_indices), GL_UNSIGNED_INT, None)
    
    #载入球体
    glBindBuffer(GL_ARRAY_BUFFER, VBO[1])
    glBufferData(GL_ARRAY_BUFFER, sphere_vertices.nbytes, sphere_vertices, GL_STATIC_DRAW)
    glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, EBO[1])
    glBufferData(GL_ELEMENT_ARRAY_BUFFER,sphere_indices.nbytes, sphere_indices, GL_STATIC_DRAW)
    #enable vertices
    glEnableVertexAttribArray(0)
    glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 24, ctypes.c_void_p(0))#从第0个数据开始，元素个数为3
    #enable color
    glEnableVertexAttribArray(1)
    glVertexAttribPointer(1, 3, GL_FLOAT, GL_FALSE, 24, ctypes.c_void_p(12))#从第1个数据开始，元素个数为3
    obj_pos = pyrr.matrix44.create_from_translation(pyrr.Vector3([xyz[0],xyz[1],xyz[2]])) #设定物体位置
    glUniformMatrix4fv(model_loc, 1, GL_FALSE, obj_pos)
    glDrawElements(GL_TRIANGLES, len(sphere_indices), GL_UNSIGNED_INT, None)
    
    glfw.swap_buffers(window)
# terminate glfw, free up allocated resources
glfw.terminate()